In [6]:
from numpy.polynomial.polynomial import Polynomial
import numpy as np
np.random.seed()

In [7]:
poly = Polynomial([1,0,0,0,0,1])

In [8]:
der = poly.deriv()
der

Polynomial([ 0.,  0.,  0.,  0.,  5.], [-1.,  1.], [-1.,  1.])

In [9]:
der(1)
x = np.array([1,2,3])
der(x)

array([   5.,   80.,  405.])

In [10]:
poly.roots()

array([-1.00000000+0.j        , -0.30901699-0.95105652j,
       -0.30901699+0.95105652j,  0.80901699-0.58778525j,
        0.80901699+0.58778525j])

In [11]:
def dist(roots,point):
    return np.abs(roots-point)

In [12]:
dists = dist(poly.roots(),x[:,np.newaxis])
np.min(dists, axis = 1)
np.argmin(dists, axis = 1 )

array([3, 3, 3], dtype=int64)

In [13]:
def get_delta(poly,deriv,point):
    return -poly(point)/deriv(point)

In [14]:
get_delta(poly,der,x)

array([-0.4       , -0.4125    , -0.60246914])

In [15]:
grid = np.array([[2.,3.,4.],[1.,-2.,3.]])
dists = grid[:,:,np.newaxis]
dists
dists = dists - poly.roots()
np.argmin(dists, axis = 2)

array([[4, 4, 4],
       [4, 4, 4]], dtype=int64)

Setup initial data

In [16]:
grid = np.array([[2.,3.,4.],[1.,-2.,3.]])
print "initial data"
print grid
done = np.ones_like(grid)
print "is done"
print done
count = np.zeros_like(grid)
print "amout of iterations"
print count
num = 0
print "roots"
print poly.roots()

initial data
[[ 2.  3.  4.]
 [ 1. -2.  3.]]
is done
[[ 1.  1.  1.]
 [ 1.  1.  1.]]
amout of iterations
[[ 0.  0.  0.]
 [ 0.  0.  0.]]
roots
[-1.00000000+0.j         -0.30901699-0.95105652j -0.30901699+0.95105652j
  0.80901699-0.58778525j  0.80901699+0.58778525j]


Run this cell X times to see how the loop works

In [17]:
dists = np.abs(grid[:,:,np.newaxis]-poly.roots())
print "distances from roots"
print dists
mins = np.min(dists,axis=2)< 1e-5
print "tolerance reached?"
print mins
done[mins] = 0
print "some are done?"
print done
grid[done == 1] += get_delta(poly,der,grid[done == 1])
print "grid updated"
print grid
count[~mins] = num
print "update count on not done"
print count[mins]
num+=1
print "which root is the closes to current solution?"
print np.argmin(dists,axis=2)

distances from roots
[[[ 3.          2.49721204  2.49721204  1.32813103  1.32813103]
  [ 4.          3.44297865  3.44297865  2.26845719  2.26845719]
  [ 5.          4.41272432  4.41272432  3.24466702  3.24466702]]

 [[ 2.          1.61803399  1.61803399  0.61803399  0.61803399]
  [ 1.          1.94008557  1.94008557  2.86985504  2.86985504]
  [ 4.          3.44297865  3.44297865  2.26845719  2.26845719]]]
tolerance reached?
[[False False False]
 [False False False]]
some are done?
[[ 1.  1.  1.]
 [ 1.  1.  1.]]
grid updated
[[ 1.5875      2.39753086  3.19921875]
 [ 0.6        -1.6125      2.39753086]]
update count on not done
[]
which root is the closes to current solution?
[[3 3 3]
 [3 0 3]]


In [18]:
def newton_matrix(poly,deriv,roots,grid,max_num=20,tolerance=1e-5):
    done = np.ones_like(grid)
    count = np.zeros_like(grid, dtype = np.float32)
    num = 0
    while np.sum(done) != 0 and num < max_num:
        dists = np.abs(grid[:,:,np.newaxis]-roots)
        mins = np.min(dists,axis=2)<tolerance
        done[mins] = 0
        grid[done == 1] += get_delta(poly,der,grid[done == 1])
        count[~mins] = num
        num+=1
    dists = np.abs(grid[:,:,np.newaxis]-roots)
    root = np.argmin(dists,axis=2)
    return count, root

In [19]:
count, root  = newton_matrix(poly,der,poly.roots(),np.array([[1.,2.+2j,3.],[-3.+1j,1.-3j,2.]]),max_num=20,tolerance=1e-3)

colors = np.array(
            [[200.,0.,100.],
             [100.,200.,0.],
             [100.,0.,200.],
             [200.,100.,0.],
             [0.,200.,100.]])
colors[root]

array([[[ 200.,    0.,  100.],
        [   0.,  200.,  100.],
        [ 200.,    0.,  100.]],

       [[ 200.,    0.,  100.],
        [ 100.,    0.,  200.],
        [ 200.,    0.,  100.]]])

In [20]:
def get_image(poly,deriv=None,roots=None,size = 200,rang = 2., colors = None, center = (0.,0.), rotation = 0.0, printing = False):

    if colors is None:
        colors = np.array(
            [[200.,0.,100.],
             [100.,200.,0.],
             [100.,0.,200.],
             [200.,100.,0.],
             [0.,200.,100.]])
    
    if deriv is None:
        deriv = poly.deriv()
    
    
    if roots is None:
        roots = poly.roots()

    grid = np.mgrid[0:size,0:size]/float(size)*rang - rang/2
    grid[0] += center[1]
    grid[1] += center[0]
    
    grid = grid[0] + 1j * grid[1]
    grid *= (np.sin(rotation) + 1j * np.cos(rotation))
    
    darken, color = newton_matrix(poly,deriv,roots,grid)
             
    img = colors[color] - darken[:,:,np.newaxis]*5.
    return img

In [49]:
from matplotlib.pyplot import imshow, imsave

plt.figure()
size = 100
img = get_image(poly, size = size, rang=4., center = (0,0), rotation= 0, printing = True)
imshow(img)
name = "obrazek {0:d}"
imsave(name.format(size),img)
plt.show()

d:\anac\lib\site-packages\IPython\kernel\__main__.py:2: RuntimeWarning: divide by zero encountered in divide
  from IPython.kernel.zmq import kernelapp as app
d:\anac\lib\site-packages\IPython\kernel\__main__.py:2: RuntimeWarning: invalid value encountered in divide
  from IPython.kernel.zmq import kernelapp as app
d:\anac\lib\site-packages\IPython\kernel\__main__.py:7: RuntimeWarning: invalid value encountered in less


In [21]:
def save_animation(poly,size,move_x,move_y,zoom,rotation,steps,stepsize):
    deriv = poly.deriv()
    roots = poly.roots()
    t = 0.0
    plt.ion()
    for i in range(steps):
        z = zoom(t)
        x = move_x(t)
        y = move_y(t)
        r = rotation(t)
        print ".",
        img = get_image(poly,deriv = deriv, roots = roots, size = size, rang = z, center = (x,y), rotation = r, printing = True)
        imsave('anim_{0:03d}'.format(i),img)
        t += stepsize
    plt.show()

In [22]:
save_animation(poly,
               256,
               lambda x: 0.01,
               lambda x: -1,
               lambda x: np.sin(x+3.14)+1.01,
               lambda x: x,
               steps = 10, stepsize = 2*3.14/100)

. . . . . . . . . .


In [23]:
def save_animation_poly_change(poly_fun,size,move_x = lambda x: 0,move_y=lambda x: 0,zoom=lambda x: 2,rotation=lambda x: 1,steps=100,stepsize=3.*3.14/100.):
    t = 0.0
    for i in range(steps):
        poly = Polynomial(poly_fun(t))
        deriv = poly.deriv()
        roots = poly.roots()
        z = zoom(t)
        x = move_x(t)
        y = move_y(t)
        r = rotation(t)
        print ".",
        img = get_image(poly,deriv = deriv, roots = roots, size = size, rang = z, center = (x,y), rotation = r, printing = True)
        imsave('anim_{0:03d}'.format(i),img)
        t += stepsize

In [24]:
def polyfun(x):
    return [1,0,2*np.cos(3*x),0,0,1]

In [61]:
save_animation_poly_change(polyfun,
                           256,
                           lambda x: 0,
                           lambda x: 0,
                           lambda x: 5,
                           lambda x: 1,
                           steps = 10,
                           stepsize = 2*3.14/100)

 . . . . . . . . . .


In [25]:
def generate_random_poly_coeffs(function_list):
    powers = np.random.randint(low=2,high=6)
    poly_coeffs = []
    for i in range(powers):
        poly_coeffs.append(np.random.choice(function_list))
    return poly_coeffs

In [26]:
class PolyFunction():
    def __init__(self,coeffs):
        self.coeffs = coeffs
        
    def __call__(self,x):
        return map(lambda y: y(x), self.coeffs)

In [27]:
p = PolyFunction([lambda x: 1,lambda x: 2,lambda x: 3])
p(1)

[1, 2, 3]

In [28]:
def save_animation_as_gif(filename):
    from subprocess import check_output, call
    call('convert anim*.png '+filename,shell=True)

In [67]:
save_animation_as_gif("a sample")

In [29]:
def random_animations(start,count, function_list=None):
    if function_list is None:
        function_list = [lambda x: 0,
                        lambda x: 1,
                        lambda x: np.sin(x),
                        lambda x: np.cos(x),
                        lambda x: 2*np.sin(x),
                        lambda x: np.sin(x/2.),
                        lambda x: 0,
                        lambda x: np.cos(2*x)]
    
    i = 0
    while i < count:
        print "Generating animation ", i
        coeffs = generate_random_poly_coeffs(function_list)
        polyfun = PolyFunction(coeffs)
        try:
            save_animation_poly_change(polyfun,
                                       256,
                                       lambda x: 0,
                                       lambda x: 0,
                                       lambda x: 5,
                                       lambda x: 1,
                                       steps = 100,
                                       stepsize = 2*3.14/100)
            save_animation_as_gif('animation' + str(start + i) + '.gif')
            i+=1
            print ""
        except Exception:
            print "Invalid polynomial generated"

        

In [69]:
random_animations(11,4)

Generating animation  0
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Generating animation  1
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Generating animation  2
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Generating animation  3
. Invalid polynomial generated
Generating animation  3
. Invalid polynomial generated
Generating animation  3
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

In [70]:
p = PolyFunction([lambda x: 1,lambda x: 0,lambda x: np.sin(x)/2+1.0, lambda x:0, lambda x:-0.5, lambda x:1])
save_animation_poly_change(p,256,steps=100)
save_animation_as_gif('interesting_flower')

d:\anac\lib\site-packages\IPython\kernel\__main__.py:2: RuntimeWarning: divide by zero encountered in divide
  from IPython.kernel.zmq import kernelapp as app
d:\anac\lib\site-packages\IPython\kernel\__main__.py:2: RuntimeWarning: invalid value encountered in divide
  from IPython.kernel.zmq import kernelapp as app
d:\anac\lib\site-packages\IPython\kernel\__main__.py:7: RuntimeWarning: invalid value encountered in less


. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .


In [31]:
from matplotlib.widgets import  RectangleSelector
from pylab import *

poly = Polynomial(p(1))
img = get_image(poly, size = 100, rang=4., center = (0,0), rotation= 0)


def onselect(eclick, erelease):
  print(' startposition : (%f, %f)' % (eclick.xdata, eclick.ydata))
  print(' endposition   : (%f, %f)' % (erelease.xdata, erelease.ydata))
  print(' used button   : ', eclick.button)

def toggle_selector(event):
    print(' Key pressed.')
    if event.key in ['Q', 'q'] and toggle_selector.RS.active:
        print(' RectangleSelector deactivated.')
        toggle_selector.RS.set_active(False)
    if event.key in ['A', 'a'] and not toggle_selector.RS.active:
        print(' RectangleSelector activated.')
        toggle_selector.RS.set_active(True)

x = arange(100)/(99.0)
y = sin(x)
fig = figure
ax = subplot(111)
ax.imshow(img)

toggle_selector.RS = RectangleSelector(ax, onselect)
connect('key_press_event', toggle_selector)
show()

d:\anac\lib\site-packages\IPython\kernel\__main__.py:2: RuntimeWarning: divide by zero encountered in divide
  from IPython.kernel.zmq import kernelapp as app
d:\anac\lib\site-packages\IPython\kernel\__main__.py:2: RuntimeWarning: invalid value encountered in divide
  from IPython.kernel.zmq import kernelapp as app
d:\anac\lib\site-packages\numpy\polynomial\_polybase.py:292: RuntimeWarning: invalid value encountered in multiply
  arg = off + scl*arg
d:\anac\lib\site-packages\IPython\kernel\__main__.py:7: RuntimeWarning: invalid value encountered in less


In [71]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time

fig = plt.figure()
ax1 = fig.add_subplot(1,1,1)

def animate(i):
    poly = Polynomial(p(i))
    img = get_image(poly, size = 100, rang=4., center = (0,0), rotation= 0)
    imshow(img)
    
ani = animation.FuncAnimation(fig, animate, interval=1000)
plt.show()

d:\anac\lib\site-packages\IPython\kernel\__main__.py:2: RuntimeWarning: divide by zero encountered in divide
  from IPython.kernel.zmq import kernelapp as app
d:\anac\lib\site-packages\IPython\kernel\__main__.py:2: RuntimeWarning: invalid value encountered in divide
  from IPython.kernel.zmq import kernelapp as app
d:\anac\lib\site-packages\IPython\kernel\__main__.py:7: RuntimeWarning: invalid value encountered in less
